In [ ]:
import numpy as np 
import cv2 as cv
import glob
import matplotlib.pyplot as plt
import os

In [ ]:
image_path_train = "output_images\\variant_0"
image_path_test = "output_images\\variant_3"

### Histogram

In [ ]:
# Histogram
def get_histogram(image):
    return [cv.calcHist([ch], [0], None, [256], [0, 256]).flatten()/ch.size for ch in cv.split(image)]

In [ ]:
test_image = cv.imread("TestImage.png")

hist_b, hist_g, hist_r = get_histogram(test_image)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(cv.cvtColor(test_image, cv.COLOR_BGR2RGB))
axes[0].axis('off')

axes[1].plot(hist_b, color='b', linewidth=1)
axes[1].plot(hist_g, color='g', linewidth=1)
axes[1].plot(hist_r, color='r', linewidth=1)

axes[1].grid()

plt.show()

### Background removal

In [ ]:

for im_path in glob.glob(f"{image_path_train}\\*.png")+glob.glob(f"{image_path_test}\\*.png"):
    if "ROI" in im_path or "mask" in im_path: continue
    
    im = cv.imread(im_path)

    hist_b, hist_g, hist_r = get_histogram(im)
    max_b, max_g, max_r = int(np.argmax(hist_b)), int(np.argmax(hist_g)), int(np.argmax(hist_r))

    #print(f"Max values: B:{max_b}, G:{max_g}, R:{max_r}")

    EPS = 5
    mask = 255-cv.inRange(im, (max_b-EPS, max_g-EPS, max_r-EPS), (max_b+EPS, max_g+EPS, max_r+EPS))

    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)))
    
    cv.imwrite(im_path.split('.')[0]+"_mask.png", mask)

    # Find contours            
    cntrs = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)[0]
    if not len(cntrs): continue
    # Merge all contours
    cntrs = cv.convexHull(np.concatenate(cntrs))

    # Bounding box and ROI
    x1, y1, w, h = cv.boundingRect(cntrs)
    x2, y2 = x1+w, y1+h

    x1 = np.clip(x1-5, 0, im.shape[1])
    y1 = np.clip(y1-5, 0, im.shape[0])
    x2 = np.clip(x2+5, 0, im.shape[1])
    y2 = np.clip(y2+5, 0, im.shape[0])
            
    cv.imwrite(im_path.split('.')[0]+"_ROI.png", im[y1:y2, x1:x2])



### Mean histogram (per class)

In [ ]:
data_image_paths = {
    clsid: glob.glob(f"{image_path_train}\\object_{clsid}*_ROI.png")
    for clsid in range(10)
}
data_mean_hist = {}

In [ ]:
for clsid, impaths in data_image_paths.items():
    images = [cv.imread(img) for img in impaths]
    histograms = np.array([get_histogram(im) for im in images], dtype=float)
    print(histograms.shape)
    if histograms.shape[0] == 0: 
        histograms = np.full((1, 3, 256), 1/256)

    b = np.mean(histograms[:,0,:], axis=0)
    g = np.mean(histograms[:,1,:], axis=0)
    r = np.mean(histograms[:,2,:], axis=0)

    fig,ax = plt.subplots(1, 1, figsize=(5, 2))
    ax.plot(b, color='b', linewidth=1)
    ax.plot(g, color='g', linewidth=1)
    ax.plot(r, color='r', linewidth=1)
    ax.grid()
    fig.tight_layout()
    plt.show()

    np.savez_compressed(f"{image_path_train}\\object_{clsid}_histogram.npz", b=b, g=g, r=r)

    data_mean_hist[clsid] = b, g, r


### Test - training data

In [ ]:
preds, labels = [], []
for im_path in glob.glob(f"{image_path_train}\\object_*_ROI.png"):

    label = int(os.path.basename(im_path).split('object_')[1].split('-')[0])

    im = cv.imread(im_path)

    im_b, im_g, im_r = get_histogram(im)
    
    # Calculate MAE
    mae = []
    for clsid, (data_b, data_g, data_r) in data_mean_hist.items():
        diff_b = np.abs(data_b-im_b)
        diff_g = np.abs(data_g-im_g)
        diff_r = np.abs(data_r-im_r)

        mae.append(np.nanmean(diff_b)+np.nanmean(diff_g)+np.nanmean(diff_r))
    
    mae = np.array(mae)/np.sum(mae)

    pred = np.argmin(mae)

    preds.append(pred)
    labels.append(label)

    #print(f"{label}, {pred}, {np.min(mae):.3f}")

In [ ]:
from sklearn.metrics import accuracy_score

print(f"Accuracy: {accuracy_score(labels, preds)*100:.1f}%")

from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Ground truth')
plt.show()

### Test - different domain

In [ ]:
preds, labels = [], []
for im_path in glob.glob(f"{image_path_test}\\object_*_ROI.png"):

    label = int(os.path.basename(im_path).split('object_')[1].split('-')[0])

    im = cv.imread(im_path)

    im_b, im_g, im_r = get_histogram(im)
    
    # Calculate MAE
    mae = []
    for clsid, (data_b, data_g, data_r) in data_mean_hist.items():
        diff_b = np.abs(data_b-im_b)
        diff_g = np.abs(data_g-im_g)
        diff_r = np.abs(data_r-im_r)

        mae.append(np.nanmean(diff_b)+np.nanmean(diff_g)+np.nanmean(diff_r))
    
    mae = np.array(mae)/np.sum(mae)

    pred = np.argmin(mae)

    preds.append(pred)
    labels.append(label)

    #print(f"{label}, {pred}, {np.min(mae):.3f}")
    

In [ ]:
print(f"Accuracy: {accuracy_score(labels, preds)*100:.1f}%")

cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Ground truth')
plt.show()

### Object detection

In [ ]:

im = cv.imread("DetectionTest.jpg")

plt.imshow(cv.cvtColor(im, cv.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

SLIDING_WINDOW_SIZE = 100
STEP = SLIDING_WINDOW_SIZE

w,h = int(np.ceil(im.shape[0]/SLIDING_WINDOW_SIZE)),int(np.ceil(im.shape[1]/SLIDING_WINDOW_SIZE))

results = np.zeros((w,h), dtype=float)

for i,y1 in enumerate(range(0, im.shape[0], STEP)):
    y2 = np.clip(y1+SLIDING_WINDOW_SIZE, 0, im.shape[0])

    for j,x1 in enumerate(range(0, im.shape[1], STEP)):        
        x2 = np.clip(x1+SLIDING_WINDOW_SIZE, 0, im.shape[1])
        
        roi = im[y1:y2, x1:x2]
        if roi.size == 0: continue

        im_b, im_g, im_r = get_histogram(roi)  

        data_b, data_g, data_r = data_mean_hist[0]
        diff_b = np.abs(data_b-im_b)
        diff_g = np.abs(data_g-im_g)
        diff_r = np.abs(data_r-im_r)

        mae = np.nanmean(diff_b)+np.nanmean(diff_g)+np.nanmean(diff_r)

        results[i,j] = mae
    
results = 255-np.round(results/np.max(results)*255.0).astype(np.uint8)
results = cv.resize(results, (im.shape[1], im.shape[0]), interpolation=cv.INTER_NEAREST)

plt.imshow(results, cmap='hot')
plt.axis('off')
plt.show()

    

### Feature engineering

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

features = []
labels = []
preds = []
for clsid, impaths in data_image_paths.items():
    for img in impaths:
        img = cv.imread(img)

        features.append(np.array(get_histogram(img), dtype=float).flatten())
        labels.append(clsid)
        
features = np.array(features)
labels = np.array(labels)

pca = PCA(n_components=24)
features = pca.fit_transform(features)

print(f"Var.: {sum(pca.explained_variance_ratio_)*100:.2f}%")

knn = KNeighborsClassifier(n_neighbors=3)

preds = knn.fit(features, labels).predict(features)

In [ ]:
print(f"Accuracy: {accuracy_score(labels, preds)*100:.1f}%")

cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Ground truth')
plt.show()